In [1]:
from ftplib import FTP
from datetime import datetime, timedelta
import zipfile
import os
import xml.etree.ElementTree as ET
import psycopg2
from psycopg2 import Error

In [2]:
# Приведение строки формата YYYY-MM-DD
# к YYYYMMDD
def normalizeDate(date):
    norm_date = str(date.year)
    
    if date.month < 10:
        norm_date += '0'
    norm_date += str(date.month)
    
    if date.day < 10:
        norm_date += '0'
    norm_date += str(date.day)
    
    return norm_date


# Возвращает сегодняшнюю дату. Формат даты: YYYYMMDD
def getCurrentDate():
    current_datetime = datetime.now()
    return normalizeDate(current_datetime)


# Возвращает множество дат за последние 15 дней
# каждая дата имеет формат: YYYYMMDD
def get15thDate():
    dates = set()
    for i in range(0, 15):
        delta = timedelta(days=i)
        current_datetime = datetime.now() - delta
        dates.add(normalizeDate(current_datetime))
    
    return dates


# Разархивация всех файлов из архива и удаление архива
def unzipFiles(abs_path_to_file, saved_dir):
    unzip_file = zipfile.ZipFile(abs_path_to_file)
    unzip_file.extractall(saved_dir) 
    unzip_file.close()
            
    os.remove(abs_path_to_file)


# Функция для скачивания файлов, загруженных на ftp-сервер
# за последние 15 дней
def downloadFilesLast15Days(ftp, saved_dir):
    files = []

    files = ftp.nlst()

    dates = get15thDate()

    with open('last_file.txt', 'w') as f:
        f.write(files[-1])
    
    for f in reversed(files):
        if f.split('_')[2] in dates:
            out = saved_dir + f
            with open(out, 'wb') as file:
                ftp.retrbinary('RETR ' + f, file.write)
            
            unzipFiles(out, saved_dir)
            
        else:
            break
    
    ftp.quit()
            

# Функция для проверки наличия новых файлов и при наличии загружать
def downloadNewestFiles(ftp, saved_dir, last_file_name):
    files = []

    files = ftp.nlst()

    with open('last_file.txt', 'w') as f:
        f.write(files[-1])
    
    for f in reversed(files):
        if f != last_file_name:
            out = saved_dir + f
            with open(out, 'wb') as file:
                ftp.retrbinary('RETR ' + f, file.write)
            
            unzipFiles(out, saved_dir)
        else:
            break 
    
    ftp.quit()

In [3]:
class CustomerRegistryData(object):
    """Класс представляет собой структуру для хранения и сохранения обновлений в БД 
    информации об организациях в реестре заказчиков"""
 
    def __init__(self, registration_number, version, full_name, ogrn, inn, kpp, legal_adress, email):
        self.registration_number = registration_number
        self.version = version
        self.full_name = full_name
        self.ogrn = ogrn
        self.inn = inn
        self.kpp = kpp
        self.legal_adress = legal_adress
        self.email = email
    
    def _show(self):
        """
        Вывод на экран всех свойств
        """
        print(self.registration_number, self.version, self.full_name, self.ogrn, 
              self.inn, self.kpp, self.legal_adress, self.email)
    
    def send2DataBase(self, connection_db, cursor_db):
        """
        Отправка данных в БД
        """
        try:
            idOrg = 0
            cursor.execute("SELECT * FROM \"Organization\" WHERE inn = \'{}\';".format(self.inn))
            record = cursor.fetchone()
            if record == None:
                insert_query = "INSERT INTO \"Organization\" (\"fullName\", ogrn, inn, kpp, addres, email) VALUES (\'{0}\', \'{1}\', \'{2}\', \'{3}\', \'{4}\', \'{5}\')".format(self.full_name, self.ogrn, self.inn, self.kpp, self.legal_adress, self.email)
                cursor.execute(insert_query)
                connection.commit()
            
                cursor.execute("SELECT * FROM \"Organization\" WHERE inn = \'{}\';".format(self.inn))
                record = cursor.fetchone()
                idOrg = record[0]
            else: 
                idOrg = record[0]
        
            cursor.execute("SELECT * FROM \"Purchase\" WHERE \"regNumb\" = \'{}\';".format(self.registration_number))
            record = cursor.fetchone()
            if record == None:
                insert_query = "INSERT INTO \"Purchase\" (version, \"regNumb\", \"idOrg\") VALUES (\'{0}\', \'{1}\', \'{2}\')".format(self.version, self.registration_number, idOrg)
                cursor.execute(insert_query)
                connection.commit()
            else:
                update_query = "UPDATE \"Purchase\"  SET version = \'{0}\'  WHERE \"regNumb\" = \'{1}\'".format(self.version, self.registration_number)
                cursor.execute(update_query)
                connection.commit()
        except (Exception, Error) as error:
            print("Ошибка при работе с PostgreSQL", error)
        
# Разбор XML-файлов и формирования массива объектов CustomerRegistryData
def parse_XML(file_path):
    tree = ET.parse(file_path)
    root = tree.getroot()
    
    tag = '{http://zakupki.gov.ru/223fz/customerRegistry/1}'
    second_tag = '{http://zakupki.gov.ru/223fz/types/1}'
    
    crd_items = []
    
    for item in root.iter(tag + 'item'):
        customer_registry_data = item.find(tag + 'customerRegistryData')
        registration_number = customer_registry_data.find(tag + 'registrationNumber').text
        version = customer_registry_data.find(tag + 'version').text
    
        customer_registry_main_info = customer_registry_data.find(tag + 'customerRegistryMainInfo')
        full_name = customer_registry_main_info.find(second_tag + 'fullName').text
        ogrn = customer_registry_main_info.find(second_tag + 'ogrn').text
        inn = customer_registry_main_info.find(second_tag + 'inn').text
        
        kpp = ''
        try:
            kpp = customer_registry_main_info.find(second_tag + 'kpp').text
        except:
            pass
        
        legal_adress = ''
        try:
            legal_adress = customer_registry_main_info.find(second_tag + 'legalAddress').text
        except:
            pass
        
        email = ''
        try:
            email = customer_registry_main_info.find(tag + 'email').text
        except:
            pass
        
        crd = CustomerRegistryData(registration_number, version, full_name, ogrn, inn, kpp, legal_adress, email)
        crd_items.append(crd)
    
    return crd_items

In [4]:
try:
    ftp = FTP('***')
    ftp.login(user='***', passwd='***')
except ftplib.all_errors as error:
    print(error)
    
try:
    ftp.cwd('out/nsi/customerRegistry/daily')
except ftplib.all_errors as error:
    print(error)

In [5]:
try:
    connection = psycopg2.connect(user="***",
                                  password="***",
                                  host="127.0.0.1",
                                  port="****",
                                  database="****")
    cursor = connection.cursor()
except (Exception, Error) as error:
    print("Ошибка при работе с PostgreSQL", error)

In [6]:
cursor.execute("SELECT * FROM \"Organization\" ;")
record = cursor.fetchone()
if record != None:
    try:
        with open('last_file.txt', 'r') as f:
            last_file = f.read()
    except:
        print('Файл не найден')
    
    if last_file == '':
        print('Пустой файл')
    
    downloadNewestFiles(ftp, 'D:\\task', last_file)
else:
    downloadFilesLast15Days(ftp, 'D:\\task')     

In [7]:
for root, dirs, files in os.walk("D:\\task", topdown=False):
    for name in files:
        items = parse_XML(os.path.join(root, name))
        for item in items:
            item.send2DataBase(connection, cursor)  

In [8]:
if connection:
        cursor.close()
        connection.close()